# Notes

Training robotics controllers using imitation learning has been a very effective approach in robotics problems where robots need to learn sequence prediction.

Imitation learning typically involves training a model to predict an expert’s behavior given encountered observations.

> Since the learner’s prediction affects future input observations/states during execution of the learned policy, this violate the crucial i.i.d. assumption made by most statistical learning approaches.

Mistakes in learner predictions can affect future observations, bringing the agent further out of the distribution of data it was trained on (because the dataset is not i.i.d).

A classifier that makes mistakes with probability $\epsilon$ may make $T^2 \epsilon$ mistakes in $T$ time-steps due to the compounding errors made from unfamiliar observations compared with expert data. This was discussed at the end of the ALVINN paper.

> We propose a new meta-algorithm for imitation learning which learns a stationary deterministic policy guaranteed to perform well under its induced distribution of states.

They have an algorithm where the policy error grows linearly with $T$ and $\epsilon$ instead of the quadratic $T^2 \epsilon$ we saw with typical imitation learning approaches.

### Preliminaries

We deal with some scenario where a model is trying to mimic the behavior of an expert.

We denote the expected immediate cost of performing some action $a$ in state $s$ as $C(s, a)$, representing the incorrectness of the action compared to the expert demonstration.

Then we can define $C_\pi(s) = \mathbb{E}_{a \sim \pi(s)}[C(s, a)]$ representing the expected immediate cost of using policy $\pi$ in $s$ by averaging the cost across it’s distribution of actions.

Then the total cost of executing policy $\pi$ for $T$ steps is given by (with $d_\pi^t$ representing the distribution of states at time $t$ under policy $\pi$):

$$
J(\pi) = \sum_{t=1}^{T} \mathbb{E}_{s \sim d_\pi^t}[C_\pi(s)]
$$

We can simplify this by using the following average visitation frequency of each state across the entire episode $d_\pi = \frac{1}{T} \sum_{t=1}^T d_\pi^t$.

Then we get the following reward function:

$$
J(\pi) = T \mathbb{E}_{s \sim d_\pi}[C_\pi(s)]
$$

Ideally, we want to optimize our policy by driving this cost $J(\pi)$ down to 0.

However, we don’t have direct access to $C(s, a)$, we only have access to the observed behavior of the expert in demonstrations.

This gives us access to the observed surrogate loss between $\pi$ and $\pi^*$ given by $\ell(s, \pi)$. In some cases, this loss may be exactly equal to $C$, for example, in the case where the model has to predict the expert action directly.

We want to find a policy $\hat{\pi}$ to minimize the surrogate loss:

$$
\hat{\pi} = \underset{\pi \in \Pi}{\arg \min} \mathbb{E}_{s \sim d_\pi} [\ell(s, \pi)]
$$

**1. Supervised Approach to Imitation Learning**

Given an error $\epsilon$ in $\pi$, we get that $J(\pi) \leq J(\pi^*) + T^2 \epsilon$.

This quadratic term gives a poor error guarantee. The policy $\pi$ will tend to perform well in the distribution of states $d_{\pi^*}$ encountered by the expert.

**2. Forward Training**

In forward training, they learn an individual policy for each time step $\pi_1, \pi_2, \pi_3, …$. By doing this, $\pi_t$ mimics $\pi^*$ on the distribution of states provided by the prior policy at time $t$.

> Hence the forward algorithm guarantees that the expected loss under the distribution of states induced by the learned policy matches the average loss during training, and hence improves performance.

This algorithm is impractical because it requires $T$ policies for $T$ time steps which is infeasible.

> Hence it can not be applied to most real-world applications.

**3. Stochastic Mixing Iterative Learning**

Another approach that solves the problem with forward training (increasing valuing it’s own policy over the expert) and achieves near-linear cost of $T\epsilon$.

### Dataset Aggregation

DAgger uses the expert policy to gather a dataset of trajectories $\mathcal{D}$ to train a policy $\hat{\pi}_2$. It then continues, adding newly collected trajectories to $\mathcal{D}$ and training a new policy $\hat{\pi}_{n+1}$ on all the trajectories collected by $\hat{\pi}_n$ and before.

![Screenshot 2024-11-06 at 3.00.44 PM.png](../../images/Screenshot_2024-11-06_at_3.00.44_PM.png)

> The intuition behind this algorithm is that over the iterations, we are building up the set of inputs that the learned policy is likely to encounter during its execution based on previous experience (training iterations).

Using DAgger, the dataset doesn’t just contain states encountered by the expert, but is trained on the entire distribution of states based on what the policy actually interacts with.

We can optionally allow querying the expert an any iteration with the following policy update specification:

$$
\pi_i = \beta_i \pi^* + (1 - \beta_i)\hat{\pi}_i
$$

They set $\beta = 1$ initially since the model shouldn’t learn from the randomly initialized expert. Then they can choose a decay function like $\beta_i = p^{i-1}$ to set an exponentially decaying probability of using the expert.

DAgger than constantly queries the expert which is one of its limitations. It essentially uses the policy to find states that the expert has yet to demonstrate. It’s main utility is in making the expert demonstration dataset more robust.

They they theoretically prove that for DAgger, the worst case overall error is linear with $T$ and $\epsilon$

### Theoretical Analysis

**1. Online Learning**

Online learning is a scenario where an algorithm first gives a policy $\pi_n$ with loss $\ell_n(\pi_n)$, and then uses this observed loss to provide a new policy $\pi_{n+1}$ with a new loss $\ell_{n+1}(\pi_{n+1})$.

This process repeats iteratively, and the model constantly integrates new data observed from a better policy.

A **no-regret algorithm** produces a sequence of polices $\pi_1, \pi_2, …, \pi_N$ so the average regret goes to 0 as the policies change.

**2. No Regret Algorithm Guarantees**

They go through a long proof to show that a no-regret algorithm gives some theoretical guarantees for the error and number of trajectories necessary for DAgger.

### Experiments

**1. Super Tux Kart**

![Screenshot 2024-11-06 at 2.40.30 PM.png](../../images/Screenshot_2024-11-06_at_2.40.30_PM.png)

> A human expert is used to provide demonstrations of the correct steering (analog joystick value in [-1,1]) for each of the observed game images.

They measure performance based on the number of falls per lap.

![Screenshot 2024-11-06 at 2.41.41 PM.png](../../images/Screenshot_2024-11-06_at_2.41.41_PM.png)

> We first observe that with the baseline supervised approach where training always occurs under the expert’s trajectories that performance does not improve as more data is collected.

More data collected from the expert doing well doesn’t help the model learn from mistakes it will inevitably make, so more data collection using the default supervised learning approach doesn’t help it.

> For DAgger, we were able to obtain a policy that never falls off
> the track after 15 iterations of training.

**2. Super Mario Bros**

> Our expert in this scenario is a near-optimal planning algorithm that has full access to the game’s internal state and can simulate exactly the consequence of future actions.

The expert can perform perfectly.

They measure performance based on the distance traveled by Mario per stage before dying, running of time, or finishing.

![Screenshot 2024-11-06 at 2.46.44 PM.png](../../images/Screenshot_2024-11-06_at_2.46.44_PM.png)

> A reason the supervised approach gets such a low score is that under the learned controller, Mario is often stuck at some location against an obstacle instead of jumping over it.

Since the expert always jumps over obstacles at a significant distance away, the controller did not learn how to get unstuck in situations where it is right next to an obstacle.

>

A cool specific example of a scenario where the model leaves the state distribution that the expert encountered an no longer knows what to do.

### Conclusion

> We show that by batching over iterations of interaction with a system, no-regret methods, including the presented DAGGER approach can provide a learning reduction with strong performance guarantees in both imitation learning and structured prediction.
